# Importação de bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
from PIL import Image
import csv
from datetime import datetime
import shutil
import piexif

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score

# Extração de metadados das imagens

In [ ]:
def extrair_metadados_imagens(caminho_pasta):
    if not os.path.isdir(caminho_pasta):
        print("O caminho especificado não é uma pasta válida.")
        return
    
    arquivos = os.listdir(caminho_pasta)
    
    metadados = []
    
    for arquivo in arquivos:
        if arquivo.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            caminho_arquivo = os.path.join(caminho_pasta, arquivo)
            
            try:
                with Image.open(caminho_arquivo) as img:
                    # Obter a data/hora da imagem
                    exif_dict = piexif.load(img.info["exif"])
                    data_hora = exif_dict['Exif'][piexif.ExifIFD.DateTimeOriginal].decode('utf-8')
                    
                    if data_hora:
                        data_hora_formatada = datetime.strptime(data_hora, "%Y:%m:%d %H:%M:%S")
                    else:
                        data_hora_formatada = "N/A"
                    
                    # Obter a localização (se disponível)
                    if piexif.GPSIFD.GPSLatitude in exif_dict['GPS'] and piexif.GPSIFD.GPSLongitude in exif_dict['GPS']:
                        # Converter a localização para graus decimais
                        lat = exif_dict['GPS'][piexif.GPSIFD.GPSLatitude]
                        lon = exif_dict['GPS'][piexif.GPSIFD.GPSLongitude]
                        lat_deg = lat[0][0] / lat[0][1] + lat[1][0] / lat[1][1] / 60 + lat[2][0] / lat[2][1] / 3600
                        lon_deg = lon[0][0] / lon[0][1] + lon[1][0] / lon[1][1] / 60 + lon[2][0] / lon[2][1] / 3600
                        
                        # Se a latitude está no hemisfério sul ou a longitude está no hemisfério oeste, tornar o valor negativo
                        if exif_dict['GPS'][piexif.GPSIFD.GPSLatitudeRef].decode('utf-8') == 'S': lat_deg = -lat_deg
                        if exif_dict['GPS'][piexif.GPSIFD.GPSLongitudeRef].decode('utf-8') == 'W': lon_deg = -lon_deg
                        
                        localizacao_formatada = f"{lat_deg}°, {lon_deg}°"
                    else:
                        localizacao_formatada = "N/A"
                    
                    metadados.append([arquivo, data_hora_formatada, localizacao_formatada])
            except Exception as e:
                print(f"Erro ao processar {arquivo}: {e}")
                metadados.append([arquivo, "N/A", "N/A"])
    
    with open(os.path.join(caminho_pasta, "info_imagens.csv"), mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["nome_arquivo", "data_hora", "localizacao"])
        writer.writerows(metadados)
    
    print("Metadados extraídos e salvos em info_imagens.csv com sucesso.")

# Adicionar o caminho da pasta
caminho_pasta = "/adicionar/caminho/pasta"
extrair_metadados_imagens(caminho_pasta)

# Gráficos das informações dos metadados 

In [ ]:
def visualizar_info_imagens(caminho_csv):
    try:
        df = pd.read_csv(caminho_csv)
    except FileNotFoundError:
        print("O arquivo CSV não foi encontrado.")
        return
    
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    
    # Gráfico de barras para distribuição das datas/horas
    plt.figure(figsize=(10, 6))
    df['data_hora'].dt.date.value_counts().sort_index().plot(kind='bar')
    plt.title('Distribuição das datas/horas das imagens')
    plt.xlabel('Data')
    plt.ylabel('Quantidade de imagens')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Gráfico de dispersão para localização das imagens (se disponível)
    df = df.dropna(subset=['localizacao'])
    if not df.empty:
        plt.figure(figsize=(10, 6))
        localizacao = df['localizacao'].str.replace('°', '').str.split(', ').apply(pd.Series)
        plt.scatter(localizacao[1].astype(float), localizacao[0].astype(float), alpha=0.5)
        plt.title('Localização das imagens')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.tight_layout()
        plt.show()
    else:
        print("Nenhuma informação de localização disponível.")

caminho_csv = "/adicionar/caminho/pasta/info_imagens.csv"
visualizar_info_imagens(caminho_csv)

# Pré-processamento das imagens

In [ ]:
def preprocessamento_imagens(caminho_pasta_origem, tamanho_novo=(200, 200)):
    if not os.path.isdir(caminho_pasta_origem):
        print("O caminho especificado não é uma pasta válida.")
        return
    
    # Criar a pasta para as imagens tratadas
    caminho_pasta_tratadas = os.path.join(caminho_pasta_origem, "imagens_tratadas")
    os.makedirs(caminho_pasta_tratadas, exist_ok=True)
    
    arquivos = os.listdir(caminho_pasta_origem)
    
    for arquivo in arquivos:
        if arquivo.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            caminho_arquivo_origem = os.path.join(caminho_pasta_origem, arquivo)
            
            with Image.open(caminho_arquivo_origem) as img:
                # Redimensionar a imagem
                img_redimensionada = img.resize(tamanho_novo)
                
                # Converter para escala de cinza
                img_cinza = img_redimensionada.convert('L')
                
                # Normalizar os valores dos pixels (0-255) para (0-1)
                img_array = np.array(img_cinza) / 255.0
                
                # Salvar a imagem tratada na pasta de imagens tratadas
                caminho_arquivo_tratado = os.path.join(caminho_pasta_tratadas, arquivo)
                img_tratada = Image.fromarray((img_array * 255).astype(np.uint8))
                img_tratada.save(caminho_arquivo_tratado)
    
    print("Pré-processamento concluído. As imagens tratadas foram salvas em 'imagens_tratadas'.")

caminho_pasta_origem = "/adicionar/caminho/pasta"
preprocessamento_imagens(caminho_pasta_origem)

# Clusterização - KMeans

In [ ]:
diretorio = "/adicionar/caminho/pasta/imagens_tratadas"

imagens = []

for filename in os.listdir(diretorio):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        img = Image.open(os.path.join(diretorio, filename))
        img = np.array(img)
        imagens.append(img)

imagens = np.array(imagens)

imagens_reshaped = imagens.reshape(len(imagens), -1)

kmeans = KMeans(n_clusters=3, random_state=42)

kmeans.fit(imagens_reshaped)

labels_kmeans = kmeans.labels_

print("Rótulos dos clusters para cada imagem:", labels_kmeans)

# Clusterização - DBSCAN

In [ ]:
# Normalizando os dados
scaler = StandardScaler()
imagens_reshaped_scaled = scaler.fit_transform(imagens_reshaped)

dbscan = DBSCAN(eps=200, min_samples=5)

dbscan.fit(imagens_reshaped_scaled)

labels_dbscan = dbscan.labels_

print("Rótulos dos clusters para cada imagem:", labels_dbscan)

# Avaliação - Coeficiente de Silhueta

In [ ]:
silhouette_kmeans = silhouette_score(imagens_reshaped, labels_kmeans)
print("Coeficiente médio de silhueta do KMeans: ", silhouette_kmeans)

silhouette_dbscan = silhouette_score(imagens_reshaped, labels_dbscan)
print("Coeficiente médio de silhueta do DBSCAN: ", silhouette_dbscan)

# Avaliação - Índice de Davies-Bouldin e Índice de Calinski-Harabasz

In [ ]:
# Calcular o Índice de Davies-Bouldin para KMeans
dbi_kmeans = davies_bouldin_score(imagens_reshaped, labels_kmeans)
print("Índice de Davies-Bouldin para KMeans: ", dbi_kmeans)

# Calcular o Índice de Davies-Bouldin para DBSCAN
dbi_dbscan = davies_bouldin_score(imagens_reshaped, labels_dbscan)
print("Índice de Davies-Bouldin para DBSCAN: ", dbi_dbscan)

# Calcular o Índice de Calinski-Harabasz para KMeans
chi_kmeans = calinski_harabasz_score(imagens_reshaped, labels_kmeans)
print("Índice de Calinski-Harabasz para KMeans: ", chi_kmeans)

# Calcular o Índice de Calinski-Harabasz para DBSCAN
chi_dbscan = calinski_harabasz_score(imagens_reshaped, labels_dbscan)
print("Índice de Calinski-Harabasz para DBSCAN: ", chi_dbscan)

# Visualização gráfica - Índice de Davies-Bouldin e Índice de Calinski-Harabasz

In [ ]:
data = {
    'Algoritmo': ['KMeans', 'KMeans', 'KMeans', 'DBSCAN', 'DBSCAN', 'DBSCAN'],
    'Métrica': ['Coeficiente de Silhueta', 'Índice de Davies-Bouldin', 'Índice de Calinski-Harabasz']*2,
    'Valor': [silhouette_kmeans, dbi_kmeans, chi_kmeans, silhouette_dbscan, dbi_dbscan, chi_dbscan]
}

df = pd.DataFrame(data)

df['Valor'] = df['Valor'].round(2)

plt.figure(figsize=(10, 6))
barplot = sns.barplot(x='Métrica', y='Valor', hue='Algoritmo', data=df)
plt.title('Comparação das Métricas de Clusterização')

for p in barplot.patches:
    barplot.annotate(format(p.get_height(), '.2f'), 
                     (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha = 'center', va = 'center', 
                     xytext = (0, 10), 
                     textcoords = 'offset points')

plt.show()

# Organização do acervo conforme o cluster

In [ ]:
df = pd.read_csv('/adicionar/caminho/pasta/info_imagens.csv')

# Criando as pastas para cada cluster
for i in range(3):
    os.makedirs(f'/adicionar/caminho/pasta/cluster_{i}', exist_ok=True)

# Iterando sobre o DataFrame e copiando as imagens para as pastas correspondentes
for i, row in df.iterrows():
    if i < len(labels_kmeans):
        cluster = labels_kmeans[i]
        shutil.copy(f"/adicionar/caminho/pasta/{row['nome_arquivo']}", f"/adicionar/caminho/pasta/cluster_{cluster}")